# Primera prueba de vectorización y clasificación básica
Se va a probar haciendo una vectorización con bag of words y clasificar usando regresión logística

In [37]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

csv = 'tweets_limpios.csv'
#Carga los tweets limpios
my_df = pd.read_csv(csv,index_col=0)
my_df.head()

,Texto,Polaridad
0,salgo de #veotv que día más largoo,NONE
1,@pauladelasheras no te libraras de ayudar meno...,NEU
2,@marodriguezb gracias mar,P
3,off pensando en el regalito sinde la que se va...,N+
4,conozco a alguien q es adicto al drama! ja ja ...,P+


Se divide el corpus en un 90% para entrenamiento y el 10% restante para pruebas. Se planea usar k-cross fold validation

In [38]:
x = my_df.Texto
y = my_df.Polaridad
SEED = 42

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.1, random_state=SEED)
print "El conjunto de entrenamiento tiene {0} registros".format(len(x_train))
print "El conjunto de prueba tiene {0} registros".format(len(x_test))

El conjunto de entrenamiento tiene 6496 registros
El conjunto de prueba tiene 722 registros


Ahora se verifica como quedó la distribución del corpus

In [39]:
#Imprimir el total de tweets por polaridad del conjunto de entrenamiento
print "Conjunto de entrenamiento:"
print y_train.value_counts()
#Imprimir el total de tweets por polaridad del conjunto de prueba
print "\nConjunto de prueba:"
print y_test.value_counts()

Conjunto de entrenamiento:
P+      1497
NONE    1346
N       1194
P       1105
N+       759
NEU      595
Name: Polaridad, dtype: int64

Conjunto de prueba:
P+      157
N       140
NONE    135
P       129
N+       87
NEU      74
Name: Polaridad, dtype: int64


## Vectorización con bag of words
Se va a hacer la vectorización con bag of words

In [43]:
#Descargar las stopwords y punkt primero con el downloader de nltk
#En la consola de Anaconda, correr: python -m nltk.downloader stopwords
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from time import time

#Método que realiza el entrenamiento y luego hace pruebas para evaluar la precisión
def accuracy_summary(pipeline, x_train, y_train, x_test, y_test):
    t0 = time()    
    #Realiza el entrenamiento
    cv = KFold(n_splits=4)
    sentiment_fit = pipeline.fit(x_train, y_train)    
    #Luego del entrenamiento, realice las predicciones
    y_pred = sentiment_fit.predict(x_test)    
    #Calcula el tiempo que duró
    train_test_time = time() - t0
    #Obtiene el accuracy de las predicciones
    accuracy = accuracy_score(y_test, y_pred)
    
    print "Exactitud: {0:.4f}%".format(accuracy*100)
    print "Duración de entrenamiento y pruebas: {0:.2f}s".format(train_test_time)
    print "-"*80
    return accuracy, train_test_time

cvec = CountVectorizer()
lr = LogisticRegression()
n_features = np.arange(10000,100001,10000)

#Método que revisa la precisión de acuerdo a la cantidad de features definidas
def nfeature_accuracy_checker(
    vectorizer=cvec, n_features=n_features, stop_words=None, ngram_range=(1, 1), classifier=lr):
    result = []
    print (classifier)
    print "\n"
    for n in n_features:
        vectorizer.set_params(stop_words=stop_words, max_features=n, ngram_range=ngram_range)
        checker_pipeline = Pipeline([
            ('vectorizer', vectorizer),
            ('classifier', classifier)
        ])
        print "Validation result for {} features".format(n)
        nfeature_accuracy,tt_time = accuracy_summary(checker_pipeline, x_train, y_train, x_test, y_test)
        result.append((n,nfeature_accuracy,tt_time))
    return result

In [46]:
#Importamos los stopwords
spanish_stopwords = stopwords.words('spanish')

print "RESULT FOR UNIGRAM WITHOUT STOP WORDS\n"
feature_result_ug = nfeature_accuracy_checker()
print "RESULT FOR UNIGRAM WITH STOP WORDS\n"
feature_result_wocsw = nfeature_accuracy_checker(stop_words=spanish_stopwords)

RESULT FOR UNIGRAM WITHOUT STOP WORDS

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


Validation result for 10000 features
Exactitud: 43.4903%
Duración de entrenamiento y pruebas: 0.64s
--------------------------------------------------------------------------------
Validation result for 20000 features
Exactitud: 43.0748%
Duración de entrenamiento y pruebas: 0.77s
--------------------------------------------------------------------------------
Validation result for 30000 features
Exactitud: 43.0748%
Duración de entrenamiento y pruebas: 0.72s
--------------------------------------------------------------------------------
Validation result for 40000 features
Exactitud: 43.0748%
Duración de entrenamiento y pruebas: 0.75s
---------------------------------------------------